In [7]:
from tensorflow.keras.layers import Layer, BatchNormalization, Conv2D, Dense, Flatten, Add, Dropout, BatchNormalization
import tensorflow as tf
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras import Input, layers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import time
import cv2

In [8]:
import cv2 as cv
import numpy as np

def opticalFlowtoMat(set_, set_idx,up_points, show = False):
    feature_params = dict( maxCorners = 100,
                        qualityLevel = 0.3,
                        minDistance = 7,
                        blockSize = 7 )
    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15, 15),
                    maxLevel = 2,
                    criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

    cap = cv.VideoCapture(f"D:/github_/ComputerVision/Assignment_5//tv_human_interactions_videos/{set_[set_idx]}") # here we should add the video dir r'D:/github_/ComputerVision/Assignment_5/__temp__.mp4'
    _, oldFrame = cap.read()

    old_gray = cv.cvtColor(oldFrame, cv.COLOR_BGR2GRAY)


    # get the values to track and find their corners 
    oldPoints = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    mask = np.zeros_like(old_gray)

    color = np.random.randint(0, 255, (100, 3))

    while(cap.isOpened()):
        ret, frame_new = cap.read()
        if ret == True:
            greyFrame_new = cv.cvtColor(frame_new, cv.COLOR_BGR2GRAY)
            newPoints, status, error = cv.calcOpticalFlowPyrLK(old_gray, greyFrame_new, oldPoints, None, **lk_params) # where p0 are the points to track

            if newPoints is not None:
                good_new = newPoints[status==1]
                good_old = oldPoints[status==1]
            
                for i, (new, old) in enumerate(zip(good_new, good_old)):
                    a, b = new.ravel()
                    c, d = old.ravel()
                    mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
                    frame = cv.circle(old_gray, (int(a), int(b)), 5, color[i].tolist(), -1)


            blank = np.ones((frame.shape[0], frame.shape[1]), dtype= mask.dtype) # This is  a black frame. 
            img = cv.add(blank, mask)

            old_gray = greyFrame_new.copy()
            oldPoints = good_new.reshape(-1, 1, 2)

        else:
            break
    if(show):
        cv.imshow('Frame',img)
        cv.waitKey(0)
    img = cv.resize(img, up_points, interpolation= cv.INTER_LINEAR)
    img2 = cv2.merge((img,img,img))

    return img2


    

In [3]:
import cv2 as cv
import numpy as np

up_width = 112
up_height = 112
up_points = (up_width, up_height)

feature_params = dict( maxCorners = 100,
                    qualityLevel = 0.3,
                    minDistance = 7,
                    blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                maxLevel = 2,
                criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

cap = cv.VideoCapture(r'D:/github_/ComputerVision/Assignment_5/__temp__.mp4') # here we should add the video dir      cap = cv.VideoCapture(f"tv_human_interactions_videos/{set_[set_idx]}") # here we should add the video dir r'D:/github_/ComputerVision/Assignment_5/__temp__.mp4'


_, oldFrame = cap.read()
# oldFrame = cv2.resize(oldFrame, up_points, interpolation= cv2.INTER_LINEAR)

old_gray = cv.cvtColor(oldFrame, cv.COLOR_BGR2GRAY)

# get the values to track and find their corners 
oldPoints = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
mask = np.zeros_like(old_gray)

color = np.random.randint(0, 255, (100, 3))

while(cap.isOpened()):
    ret, frame_new = cap.read()

    if ret == True:
        # frame_new = cv2.resize(frame_new, up_points, interpolation= cv2.INTER_LINEAR)
        greyFrame_new = cv.cvtColor(frame_new, cv.COLOR_BGR2GRAY)
        newPoints, status, error = cv.calcOpticalFlowPyrLK(old_gray, greyFrame_new, oldPoints, None, **lk_params) # where p0 are the points to track

        if newPoints is not None:
            good_new = newPoints[status==1]
            good_old = oldPoints[status==1]
        
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
                frame = cv.circle(old_gray, (int(a), int(b)), 5, color[i].tolist(), -1)


        blank = np.ones((frame.shape[0], frame.shape[1]), dtype= mask.dtype) # This is  a black frame. 
        img = cv.add(blank, mask)

        old_gray = greyFrame_new.copy()
        oldPoints = good_new.reshape(-1, 1, 2)

    else:
        break
img = cv.resize(img, up_points, interpolation= cv.INTER_LINEAR)
cv.imshow('Frame',img)
cv.waitKey(0)


#112 ,112 ,3





In [53]:
width =112
height = 112

def get_model(input_shape, outputShape,opt, lossFunc):
    x = Input(shape=input_shape)
    h = Conv2D(32, padding='same', kernel_size=(3,3), activation='relu')(x)
    h = Conv2D(32, padding='same', kernel_size=(3,3), activation='relu')(x)
    h = MaxPooling2D(pool_size=(2,2))(h)
    h = Conv2D(64, padding='same', kernel_size=(3,3), activation='relu')(h)
    h = Conv2D(64, padding='same', kernel_size=(3,3), activation='relu')(h)
    h = MaxPooling2D(pool_size=(2,2))(h)
    h = Conv2D(128, kernel_size=(3,3), activation='relu')(h)
    h = Conv2D(128, kernel_size=(3,3), activation='relu')(h)
    h = MaxPooling2D(pool_size=(2,2))(h)
    h = Flatten()(h)
    h = Dense(128, activation='relu')(h)
    h = Dropout(.5)(h)
    output = Dense(outputShape, activation='softmax')(h)

    model = tf.keras.Model(inputs=x, outputs=output)
    
    model.compile(optimizer=opt, # 'adam'
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # 
             metrics=['accuracy'])
    return model
 
benchmark_model = get_model((width, height, 3), 4,'adam', 'categorical_crossentropy') # len(train_labels)
benchmark_model.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 112, 112, 3)]     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 56, 56, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 64)       0         
 2D)                                                         

(112, 112)

In [ ]:
# Set 1
Hand_shake  = {2,14,15,16,18,19,20,21,24,25,26,27,28,32,40,41,42,43,44,45,46,47,48,49,50}
High_five = {1,6,7,8,9,10,11,12,13,23,24,25,27,28,29,30,31,32,33,34,35,44,45,47,48}
Hug = {2,3,4,11,12,15,16,17,18,20,21,27,29,30,31,32,33,34,35,36,42,44,46,49,50}	
Kiss= {1,7,8,9,10,11,12,13,14,16,17,18,22,23,24,26,29,31,35,36,38,39,40,41,42}

set_1 = [Hand_shake, High_five, Hug, Kiss]
# Set 2
Hand_shake_2 = {1,3,4,5,6,7,8,9,10,11,12,13,17,22,23,29,30,31,33,34,35,36,37,38,39}
High_five_2={2,3,4,5,14,15,16,17,18,19,20,21,22,26,36,37,38,39,40,41,42,43,46,49,50}
Hug_2= {1,5,6,7,8,9,10,13,14,19,22,23,24,25,26,28,37,38,39,40,41,43,45,47,48}
Kiss_2 = {2,3,4,5,6,15,19,20,21,25,27,28,30,32,33,34,37,43,44,45,46,47,48,49,50}

set_2 = [Hand_shake_2, High_five_2, Hug_2, Kiss_2]


set_name = ["Hand_shake", "High_five", "Hug", "Kiss"]

dic = {}
labelList_set1= [None] * 51
for idx, class_ in enumerate(set_1):
    for num in class_:
        if(labelList_set1[int(num)] is not None):
            labelList_set1[int(num)].append(idx)
        else:
            labelList_set1[int(num)] = [idx]
dic = {}
labelList_set2= [None] * 51
for idx, class_ in enumerate(set_2):
    for num in class_:
        if(labelList_set2[int(num)] is not None):
            labelList_set2[int(num)].append(idx)
        else:
            labelList_set2[int(num)] = [idx]
            
z1 = np.zeros((51,4))
z2 = np.zeros((51,4))
idx =0
for l1, l2 in zip(labelList_set1, labelList_set2) :
    if(l1 is not None):
        z1[idx,l1] = 1
    if(l2 is not None):
        z2[idx, l2] = 1
    idx = idx+1


In [ ]:
z1_1 = np.zeros((1,51))
z1_2 = np.zeros((1,51))
z1_3 = np.zeros((1,51))
z1_4 = np.zeros((1,51))

z1l = [z1_1, z1_2,z1_3, z1_4]


z2_1 = np.zeros((1,51))
z2_2 = np.zeros((1,51))
z2_3 = np.zeros((1,51))
z2_4 = np.zeros((1,51))

z2l = [z2_1, z2_2,z2_3, z2_4]

In [54]:

for idx, row in enumerate(z1):
    ss = [i for i, j in enumerate(row) if j == 1]
    for i in ss:
        z1l[i][:,idx] = 1


for idx, row in enumerate(z2):
    ss = [i for i, j in enumerate(row) if j == 1]
    for i in ss:
        z2l[i][:,idx] = 1

        

print(ss)

[1, 3]


In [55]:
from moviepy.editor import *

video_no = 5  # change this to a number between [0, 100] and you can see a different training video from Set 2

clip=VideoFileClip(f'TV-HI/tv_human_interactions_videos/{set_2[video_no]}')
print(f'\n\nA video with the label - {set_2_label[video_no]}\n')
clip.ipython_display(width=380)

25.0

In [ ]:
# Set 1
Hand_shake  = {2,14,15,16,18,19,20,21,24,25,26,27,28,32,40,41,42,43,44,45,46,47,48,49,50}
High_five = {1,6,7,8,9,10,11,12,13,23,24,25,27,28,29,30,31,32,33,34,35,44,45,47,48}
Hug = {2,3,4,11,12,15,16,17,18,20,21,27,29,30,31,32,33,34,35,36,42,44,46,49,50}	
Kiss= {1,7,8,9,10,11,12,13,14,16,17,18,22,23,24,26,29,31,35,36,38,39,40,41,42}

set_1 = [Hand_shake, High_five, Hug, Kiss]
# Set 2
Hand_shake_2 = {1,3,4,5,6,7,8,9,10,11,12,13,17,22,23,29,30,31,33,34,35,36,37,38,39}
High_five_2={2,3,4,5,14,15,16,17,18,19,20,21,22,26,36,37,38,39,40,41,42,43,46,49,50}
Hug_2= {1,5,6,7,8,9,10,13,14,19,22,23,24,25,26,28,37,38,39,40,41,43,45,47,48}
Kiss_2 = {2,3,4,5,6,15,19,20,21,25,27,28,30,32,33,34,37,43,44,45,46,47,48,49,50}

set_2 = [Hand_shake_2, High_five_2, Hug_2, Kiss_2]


set_name = ["Hand_shake", "High_five", "Hug", "Kiss"]


25.0

In [4]:
#SET 1:
set_1_ind =[[2,14,15,16,18,19,20,21,24,25,26,27,28,32,40,41,42,43,44,45,46,47,48,49,50],
[ 1,6,7,8,9,10,11,12,13,23,24,25,27,28,29,30,31,32,33,34,35,44,45,47,48],
[2,3,4,11,12,15,16,17,18,20,21,27,29,30,31,32,33,34,35,36,42,44,46,49,50],[ 1,7,8,9,10,11,12,13,14,16,17,18,22,23,24,26,29,31,35,36,38,39,40,41,42]]


#SET 2:
set_2_ind = [[1,3,4,5,6,7,8,9,10,11,12,13,17,22,23,29,30,31,33,34,35,36,37,38,39], [2,3,4,5,14,15,16,17,18,19,20,21,22,26,36,37,38,39,40,41,42,43,46,49,50], 
[1,5,6,7,8,9,10,13,14,19,22,23,24,25,26,28,37,38,39,40,41,43,45,47,48], [2,3,4,5,6,15,19,20,21,25,27,28,30,32,33,34,37,43,44,45,46,47,48,49,50]]

set_name = ["handShake", "highFive", "hug", "kiss"]


set_1 = [f'{set_name[c]}_{i:04d}.avi' for c in range(len(set_name)) for i in set_1_ind[c]]
set_1_label =[f'{set_name[c]}' for c in range(len(set_name)) for i in set_1_ind[c]]

set_2 = [f'{set_name[c]}_{i:04d}.avi' for c in range(len(set_name)) for i in set_2_ind[c]]
set_2_label =[f'{set_name[c]}' for c in range(len(set_name)) for i in set_2_ind[c]]

set_2 = [f'{set_name[c]}_{i:04d}.avi' for c in range(len(set_name)) for i in set_2_ind[c]]
set_2_label =[f'{set_name[c]}' for c in range(len(set_name)) for i in set_2_ind[c]]

# note that we ignore the negative / not found founds

print(f'Set 2 to be used for train and validation \n\t{len(set_2)}/{set_2} ')


Set 2 to be used for train and validation 
	100/['handShake_0001.avi', 'handShake_0003.avi', 'handShake_0004.avi', 'handShake_0005.avi', 'handShake_0006.avi', 'handShake_0007.avi', 'handShake_0008.avi', 'handShake_0009.avi', 'handShake_0010.avi', 'handShake_0011.avi', 'handShake_0012.avi', 'handShake_0013.avi', 'handShake_0017.avi', 'handShake_0022.avi', 'handShake_0023.avi', 'handShake_0029.avi', 'handShake_0030.avi', 'handShake_0031.avi', 'handShake_0033.avi', 'handShake_0034.avi', 'handShake_0035.avi', 'handShake_0036.avi', 'handShake_0037.avi', 'handShake_0038.avi', 'handShake_0039.avi', 'highFive_0002.avi', 'highFive_0003.avi', 'highFive_0004.avi', 'highFive_0005.avi', 'highFive_0014.avi', 'highFive_0015.avi', 'highFive_0016.avi', 'highFive_0017.avi', 'highFive_0018.avi', 'highFive_0019.avi', 'highFive_0020.avi', 'highFive_0021.avi', 'highFive_0022.avi', 'highFive_0026.avi', 'highFive_0036.avi', 'highFive_0037.avi', 'highFive_0038.avi', 'highFive_0039.avi', 'highFive_0040.avi', 'h

In [55]:
def return_processed_middle_frame(set_, set_idx):


    cap = cv2.VideoCapture(f"tv_human_interactions_videos/{set_[set_idx]}")
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    middle_frame = int(length / 2)
    cap.set(1, middle_frame)
    ret, frame = cap.read()
    frame = frame / 255.0
    frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
    return frame

def preprocess_labels(labels):
    train_labels_names = set(labels)
    #train_labels_nums = [i for i in range(len(train_labels_names))]
    train_labels_y = []

    train_labels_names = list(train_labels_names)

    for class_name in labels:
        i = train_labels_names.index(class_name)
        train_labels_y.append(i)

    return np.array(train_labels_y)

In [56]:


width = 112
height = 112
dim = (width, height)

'''Obtaining middle frames of test and train sets'''
test_images_TV = [opticalFlowtoMat(set_1, set_idx, dim) for set_idx in range(len(set_1))]
test_images_TV = np.array(test_images_TV)
train_images_TV = [opticalFlowtoMat(set_2, set_idx,  dim) for set_idx in range(len(set_2))]
train_images_TV = np.array(train_images_TV)




In [56]:
test_labels_y_TV = preprocess_labels(set_1_label)
train_labels_y_TV = preprocess_labels(set_2_label)

In [66]:
benchmark_model.fit(train_images_TV,train_labels_y_TV , epochs=20, batch_size=32)

Epoch 1/20
4/4 [==============================] - 1s 24ms/step - loss: 0.2453 - accuracy: 0.9000
Epoch 2/20
4/4 [==============================] - 0s 24ms/step - loss: 0.2463 - accuracy: 0.9300
Epoch 3/20
4/4 [==============================] - 0s 23ms/step - loss: 0.2325 - accuracy: 0.9200
Epoch 4/20
4/4 [==============================] - 0s 24ms/step - loss: 0.1299 - accuracy: 0.9700
Epoch 5/20
4/4 [==============================] - 0s 23ms/step - loss: 0.1168 - accuracy: 0.9400
Epoch 6/20
4/4 [==============================] - 0s 25ms/step - loss: 0.0630 - accuracy: 0.9800
Epoch 7/20
4/4 [==============================] - 0s 24ms/step - loss: 0.0519 - accuracy: 0.9700
Epoch 8/20
4/4 [==============================] - 0s 27ms/step - loss: 0.0975 - accuracy: 0.9700
Epoch 9/20
4/4 [==============================] - 0s 22ms/step - loss: 0.1259 - accuracy: 0.9500
Epoch 10/20
4/4 [==============================] - 0s 21ms/step - loss: 0.0418 - accuracy: 1.0000
Epoch 11/20
4/4 [============

In [15]:
set_1[0]

'Hand_shake_0002.avi'

In [38]:
d = cv.VideoCapture(f"D:/github_/ComputerVision/Assignment_5/tv_human_interactions_videos/{set_1[2]}") # here we should add the video dir r'D:/github_/ComputerVision/Assignment_5/__temp__.mp4'


In [39]:
print(d.read()[0])

True


In [73]:
def cal(olfimg, img):

    feature_params = dict( maxCorners = 100,
                    qualityLevel = 0.3,
                    minDistance = 7,
                    blockSize = 7 )
    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15, 15),
                    maxLevel = 2,
                    criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

    cap = cv.VideoCapture(r'D:/github_/ComputerVision/Assignment_5/__temp__.mp4') # here we should add the video dir      cap = cv.VideoCapture(f"tv_human_interactions_videos/{set_[set_idx]}") # here we should add the video dir r'D:/github_/ComputerVision/Assignment_5/__temp__.mp4'
    _, oldFrame = cap.read()

    r_o,g_o,b_o = cv2.split(oldFrame)

    mask_r = np.zeros_like(r_o)
    mask_g = np.zeros_like(g_o)
    mask_b = np.zeros_like(b_o)


    oldPoints_r = cv.goodFeaturesToTrack(r_o, mask = None, **feature_params)
    oldPoints_g = cv.goodFeaturesToTrack(g_o, mask = None, **feature_params)
    oldPoints_b = cv.goodFeaturesToTrack(b_o, mask = None, **feature_params)

    r_n,g_n,b_n = cv2.split(img)

    newPoints_r, status_r, error_r = cv.calcOpticalFlowPyrLK(r_o, r_n, oldPoints_r, None, **lk_params) # where p0 are the points to track
    newPoints_b, status_r, error_r = cv.calcOpticalFlowPyrLK(g_o, g_n, oldPoints_g, None, **lk_params) # where p0 are the points to track
    newPoints_g, status_r, error_r = cv.calcOpticalFlowPyrLK(b_o, b_n, oldPoints_b, None, **lk_params) # where p0 are the points to track
    while(cap.isOpened()):
        ret, frame_new = cap.read()

        if ret == True:

            # check for r
            if newPoints_r is not None:
                good_new_r = newPoints_r[status==1]
                good_old_r = oldPoints_r[status==1]
                for (old, new) in zip(good_old_r, good_new_r):
                    a,b = new.ravel()
                    c,d = old.ravel()
                    mask_r = cv.line(mask,int(a), int(b), int(c), int(d),5,(255,0,0), 2)
                        
            # check for g

            if newPoints_g is not None:
                good_new_g = newPoints_g[status == 1]
                good_old_g = oldPoints_g[status ==1]
                for (old, new) in zip(good_old_g, good_new_g):
                    a,b = new.ravel()
                    c,d = old.ravel()
                    mask_g = cv.line(mask,int(a), int(b), int(c), int(d),5,(0,255,0), 2)
        
            # check for b
            if newPoints_b is not None:
                good_new_b = newPoints_b[status==1]
                good_old_b = oldPoints_b[status==1]

                for (old, new) in zip(good_old_b, good_new_b):
                    a,b = new.ravel()
                    c,d = old.ravel()
                    mask_b = cv.line(mask,int(a), int(b), int(c), int(d),5,(0,0,255), 2)


            blank_r = np.ones((mask_r.shape[0], mask_r.shape[1]), dtype= mask.dtype) # This is  a black frame. 
            blank_g = np.ones((mask_g.shape[0], mask_g.shape[1]), dtype= mask.dtype) # This is  a black frame. 
            blank_b = np.ones((mask_b.shape[0], mask_b.shape[1]), dtype= mask.dtype) # This is  a black frame. 

            img_r = cv.add(blank_r, mask_r)
            img_g = cv.add(blank_g, mask_g)
            img_b = cv.add(blank_b, mask_b)

            oldPoints_r= newPoints_r.copy()
            oldPoints_g= newPoints_g.copy()
            oldPoints_b= newPoints_b.copy()

            # old_gray = greyFrame_new.copy()

            oldPoints_r= newPoints_r.reshape(-1, 1, 2)
            oldPoints_g= newPoints_g.reshape(-1, 1, 2)
            oldPoints_b= newPoints_b.reshape(-1, 1, 2)
            # oldPoints = good_new.reshape(-1, 1, 2)
        else:
            break

        
    img = cv.merge((img_r,img_g, img_b ))
    return img

    

In [ ]:

feature_params = dict( maxCorners = 100,
                qualityLevel = 0.3,
                minDistance = 7,
                blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                maxLevel = 2,
                criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

cap = cv.VideoCapture(r'D:/github_/ComputerVision/Assignment_5/__temp__.mp4') # here we should add the video dir      cap = cv.VideoCapture(f"tv_human_interactions_videos/{set_[set_idx]}") # here we should add the video dir r'D:/github_/ComputerVision/Assignment_5/__temp__.mp4'
_, oldFrame = cap.read()
print(oldFrame.shape)
r_o,g_o,b_o = cv2.split(oldFrame)

mask_r = np.zeros_like(r_o)
mask_g = np.zeros_like(g_o)
mask_b = np.zeros_like(b_o)


oldPoints_r = cv.goodFeaturesToTrack(r_o, mask = None, **feature_params)
oldPoints_g = cv.goodFeaturesToTrack(g_o, mask = None, **feature_params)
oldPoints_b = cv.goodFeaturesToTrack(b_o, mask = None, **feature_params)

while(cap.isOpened()):
    ret, frame_new = cap.read()

    if ret == True:

        r_n,g_n,b_n = cv2.split(frame_new)

        newPoints_r, status_r, error_r = cv.calcOpticalFlowPyrLK(r_o, r_n, oldPoints_r, None, **lk_params) # where p0 are the points to track
        newPoints_b, status_r, error_r = cv.calcOpticalFlowPyrLK(g_o, g_n, oldPoints_g, None, **lk_params) # where p0 are the points to track
        newPoints_g, status_r, error_r = cv.calcOpticalFlowPyrLK(b_o, b_n, oldPoints_b, None, **lk_params) # where p0 are the points to track

        print("R")

        # check for r
        if newPoints_r is not None:
            print(newPoints_r)
            good_new_r = newPoints_r[status==1]
            good_old_r = oldPoints_r[status==1]
            for (old, new) in zip(good_old_r, good_new_r):
                a,b = new.ravel()
                c,d = old.ravel()
                mask_r = cv.line(mask,int(a), int(b), int(c), int(d),5,(255,0,0), 2)
        print("G")
        # check for g

        if newPoints_g is not None:
            good_new_g = newPoints_g[status == 1]
            good_old_g = oldPoints_g[status ==1]
            for (old, new) in zip(good_old_g, good_new_g):
                a,b = new.ravel()
                c,d = old.ravel()
                mask_g = cv.line(mask,int(a), int(b), int(c), int(d),5,(0,255,0), 2)
        print("B")

        # check for b
        if newPoints_b is not None:
            good_new_b = newPoints_b[status==1]
            good_old_b = oldPoints_b[status==1]

            for (old, new) in zip(good_old_b, good_new_b):
                a,b = new.ravel()
                c,d = old.ravel()
                mask_b = cv.line(mask,int(a), int(b), int(c), int(d),5,(0,0,255), 2)


        print("Round 1")

        blank_r = np.ones((mask_r.shape[0], mask_r.shape[1]), dtype= mask.dtype) # This is  a black frame. 
        blank_g = np.ones((mask_g.shape[0], mask_g.shape[1]), dtype= mask.dtype) # This is  a black frame. 
        blank_b = np.ones((mask_b.shape[0], mask_b.shape[1]), dtype= mask.dtype) # This is  a black frame. 

        img_r = cv.add(blank_r, mask_r)
        img_g = cv.add(blank_g, mask_g)
        img_b = cv.add(blank_b, mask_b)

        oldPoints_r= newPoints_r.copy()
        oldPoints_g= newPoints_g.copy()
        oldPoints_b= newPoints_b.copy()

        # old_gray = greyFrame_new.copy()

        oldPoints_r= newPoints_r.reshape(-1, 1, 2)
        oldPoints_g= newPoints_g.reshape(-1, 1, 2)
        oldPoints_b= newPoints_b.reshape(-1, 1, 2)
        # oldPoints = good_new.reshape(-1, 1, 2)
    else:
        break

    
img = cv.merge((img_r,img_g, img_b ))

cv.imshow(img)
cv.waitKey(0)

In [86]:


import cv2 as cv
import numpy as np

up_width = 112
up_height = 112
up_points = (up_width, up_height)

feature_params = dict( maxCorners = 100,
                    qualityLevel = 0.3,
                    minDistance = 7,
                    blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                maxLevel = 2,
                criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

cap = cv.VideoCapture(r'D:/github_/ComputerVision/Assignment_5/__temp__.mp4') # here we should add the video dir      cap = cv.VideoCapture(f"tv_human_interactions_videos/{set_[set_idx]}") # here we should add the video dir r'D:/github_/ComputerVision/Assignment_5/__temp__.mp4'


_, oldFrame = cap.read()
# oldFrame = cv2.resize(oldFrame, up_points, interpolation= cv2.INTER_LINEAR)

old_gray = cv.cvtColor(oldFrame, cv.COLOR_BGR2GRAY)

# get the values to track and find their corners 
oldPoints = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
mask = np.zeros_like(old_gray)

color = np.random.randint(0, 255, (100, 3))

while(cap.isOpened()):
    ret, frame_new = cap.read()

    if ret == True:
        # frame_new = cv2.resize(frame_new, up_points, interpolation= cv2.INTER_LINEAR)
        greyFrame_new = cv.cvtColor(frame_new, cv.COLOR_BGR2GRAY)
        newPoints, status, error = cv.calcOpticalFlowPyrLK(old_gray, greyFrame_new, oldPoints, None, **lk_params) # where p0 are the points to track

        if newPoints is not None:
            print(newPoints)
            break
            
            good_new = newPoints[status==1]
            good_old = oldPoints[status==1]
        
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
                frame = cv.circle(old_gray, (int(a), int(b)), 5, color[i].tolist(), -1)


        blank = np.ones((frame.shape[0], frame.shape[1]), dtype= mask.dtype) # This is  a black frame. 
        img = cv.add(blank, mask)

        old_gray = greyFrame_new.copy()
        oldPoints = good_new.reshape(-1, 1, 2)

    else:
        break
img = cv.resize(img, up_points, interpolation= cv.INTER_LINEAR)
cv.imshow('Frame',img)
cv.waitKey(0)


#112 ,112 ,3





[[[599.       301.00003 ]]

 [[312.84256  294.95316 ]]

 [[291.83835  255.8965  ]]

 [[516.       236.      ]]

 [[278.87076  228.98538 ]]

 [[438.00256  162.99986 ]]

 [[603.0002   309.00003 ]]

 [[295.838    269.9642  ]]

 [[328.0001    55.999844]]

 [[358.5343   302.9304  ]]

 [[334.00015   51.99962 ]]

 [[344.95374  113.9961  ]]

 [[321.00003   49.000065]]

 [[311.902    310.978   ]]

 [[476.81186  340.27673 ]]

 [[320.0003   162.99925 ]]

 [[517.99994  290.99976 ]]

 [[406.95242  101.905785]]

 [[400.55737  272.274   ]]

 [[355.61786  308.2187  ]]

 [[310.        31.      ]]]


Error: Canceled future for execute_request message before replies were done

In [ ]:
def return_processed_middle_frame(set_, set_idx):
    width = 112
    height = 112
    dim = (width, height)

    cap = cv2.VideoCapture(f"tv_human_interactions_videos/{set_[set_idx]}")
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    middle_frame = int(length / 2)
    cap.set(1, middle_frame)
    ret, frame = cap.read()
    frame = frame / 255.0
    frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
    return frame

In [ ]:
def op(frame1,frame2):
    prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)

    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

    return bgr



In [59]:
import numpy as np
import cv2 as cv

def denseOpticalFlow(set_, set_idx, num_frames,dim ,show = False):


    cap = cv2.VideoCapture(f"tv_human_interactions_videos/{set_[set_idx]}")
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    middle_frame = int(length / 2)
    cap.set(1, middle_frame)
    ret, frame1 = cap.read()
    
    prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[..., 1] = 255
    i =0

    out = np.zeros((dim[0],dim[1],3,num_frames))
    while(i<num_frames):
        ret, frame2 = cap.read()
        if not ret:
            print('No frames grabbed!')
            break
        next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
        flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = ang*180/np.pi/2
        hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
        bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
        
        if(num_frames>1):
            bgr = cv2.resize(bgr, dim, interpolation=cv2.INTER_AREA)
            out[:,:,:,i] = bgr
        i+=1
        if(show):
            cv.imshow("Frame", bgr)
            cv.waitKey(50)
    # print(f"Out: {i}")

    bgr = cv2.resize(bgr, dim, interpolation=cv2.INTER_AREA)
    if(i>1):
        return out
        
    return bgr

train_images_TV = [denseOpticalFlow(set_2, set_idx,1, (112,112), show = False) for set_idx in range(len(set_2))]
train_images_TV = np.array(train_images_TV)

test_images_TV = [denseOpticalFlow(set_1, set_idx,1, (112,112),show = False) for set_idx in range(len(set_1))]
test_images_TV = np.array(test_images_TV)


(100, 112, 112, 3)


In [31]:
cap = cv.VideoCapture(r'D:/github_/ComputerVision/Assignment_5/__temp__.mp4') # here we should add the video dir      cap = cv.VideoCapture(f"tv_human_interactions_videos/{set_[set_idx]}") # here we should add the video dir r'D:/github_/ComputerVision/Assignment_5/__temp__.mp4'
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv.imshow('frame2', bgr)
    k = cv.waitKey(30) & 0xff

No frames grabbed!


In [63]:
benchmark_model.fit(train_images_TV,train_labels_y_TV , epochs=20, batch_size=32)

benchmark_model, test_acc = benchmark_model.evaluate(test_images_TV, test_labels_y_TV, verbose=2)
print(test_acc)

AttributeError: 'float' object has no attribute 'fit'